# Pipeline de ETL com Python

**Contexto**: Foi passada a tarefa de envolver os clientes de maneira mais personalizada. O objetivo é criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Tarefa**:
- Você recebeu uma planilha em formato CSV com os dados de 300 clientes do banco, o dados são:


```
ID do usuário
Nome do usuário
Idade do usuário
Agencia
Conta
Saldo da conta
Cartão
Limite do cartão
```
- O trabalho será gerar para cada cliente, conforme sua faixa etária, uma mensagem do produto destinado a faixa de idade.

```
18 ~ 29 anos --> Importancia de investimentos
30 ~ 50 anos --> Importancia do seguro de vida
```

- Uma vez que as mensagens para cada cliente esteja pronta, deve ser salva junto aos dados forncedidos em um banco de dados no formato json.



## Bibliotecas

In [1]:
import pandas as pd
from tinydb import TinyDB

## Carregando base de dados

In [26]:
dataset = pd.read_csv('./dados_csv/database.csv')
dataset.head(10)

,Id,Nome,Idade,Agencia,Conta,Saldo,Cartao,Limite
0,1,Daniel Farmer,24,1001,93728,863.59,**** **** **** 9864,386.30
1,2,Marcus Lamb,37,1001,80952,1309.10,**** **** **** 1507,840.15
2,3,Emily Woods,41,1001,70464,970.00,**** **** **** 4691,903.74
3,4,Cameron Davis,50,1001,65039,553.90,**** **** **** 6047,865.23
4,5,Shari Williams,43,1001,42287,1526.66,**** **** **** 7182,466.63
5,6,Barbara Hayden,30,1001,6972,615.26,**** **** **** 8714,514.79
6,7,Thomas Joseph,49,1001,66167,562.57,**** **** **** 9755,370.73
7,8,William Collins,39,1001,59890,1342.15,**** **** **** 9831,409.81
8,9,Patricia Bryan,27,1001,84573,1831.18,**** **** **** 2002,480.95
9,10,Francisco Fleming,21,1001,47842,826.81,**** **** **** 5686,306.60


In [19]:
# Dimensões da base de dados
dataset.shape

(200, 8)

## Enriquecimento da base de dados

In [20]:
# Função para gerar mensagens personalizadas com base na idade do usuário
def gerar_mensagem_personalizada(idade, usuario):
    if idade >= 18 and idade < 30:
        mensagem = f"{usuario}, prepare seu futuro investindo com retorno garantido."
    else:
        mensagem = f"{usuario}, proteja-se com nosso seguro de vida."

    return mensagem

noticias = pd.DataFrame(columns=['Id', 'Nome', 'Mensagem'])

# Copiar os valores das colunas 'Id' e 'Nome' do DataFrame 'dataset' para 'noticias'
noticias['Id'] = dataset['Id']
noticias['Nome'] = dataset['Nome']

# Criar uma lista de mensagens personalizadas com base nas idades
idades = dataset['Idade'].tolist()
usuarios = dataset['Nome'].tolist()
mensagens = []

for i in range(len(idades)):
    mensagem = gerar_mensagem_personalizada(idades[i], usuarios[i])
    mensagens.append(mensagem)

# Adicionar a lista de mensagens ao DataFrame 'noticias'
noticias['Mensagem'] = mensagens

In [21]:
# Visualizando a base de noticias
noticias.head(10)

,Id,Nome,Mensagem
0,1,Daniel Farmer,"Daniel Farmer, prepare seu futuro investindo c..."
1,2,Marcus Lamb,"Marcus Lamb, proteja-se com nosso seguro de vida."
2,3,Emily Woods,"Emily Woods, proteja-se com nosso seguro de vida."
3,4,Cameron Davis,"Cameron Davis, proteja-se com nosso seguro de ..."
4,5,Shari Williams,"Shari Williams, proteja-se com nosso seguro de..."
5,6,Barbara Hayden,"Barbara Hayden, proteja-se com nosso seguro de..."
6,7,Thomas Joseph,"Thomas Joseph, proteja-se com nosso seguro de ..."
7,8,William Collins,"William Collins, proteja-se com nosso seguro d..."
8,9,Patricia Bryan,"Patricia Bryan, prepare seu futuro investindo ..."
9,10,Francisco Fleming,"Francisco Fleming, prepare seu futuro investin..."


In [22]:
# Juntando os DataFrames usando o Nome como chave
df_mesclado = pd.merge(dataset, noticias, on='Nome')

# Criando uma lista de dicionários
lista_resultado = []
for _, linha in df_mesclado.iterrows():
    entrada = {
        'Id': linha['Id_x'],
        'Nome': linha['Nome'],
        'Idade': linha['Idade'],
        'Agencia': linha['Agencia'],
        'Conta': linha['Conta'],
        'Saldo': linha['Saldo'],
        'Cartao': linha['Cartao'],
        'Limite': linha['Limite'],
        'noticias':
         [
            {
            'Id': linha['Id_y'],
            'Mensagem': linha['Mensagem']
        }
      ]
    }
    lista_resultado.append(entrada)

# Convertendo a lista em um dicionário
resultado_dicionario = {cliente['Id']: cliente for cliente in lista_resultado}


## Salvando dados no TinyDB em json

In [23]:
# Cria o banco de dados
database_json = TinyDB('databasej.json')

In [24]:
# Inseri os dados no banco
database_json.insert(resultado_dicionario)

1

In [25]:
# Consulta os dados do banco
database_json.all()

[{'1': {'Id': 1,
   'Nome': 'Daniel Farmer',
   'Idade': 24,
   'Agencia': 1001,
   'Conta': 93728,
   'Saldo': 863.59,
   'Cartao': '**** **** **** 9864',
   'Limite': 386.3,
   'noticias': [{'Id': 1,
     'Mensagem': 'Daniel Farmer, prepare seu futuro investindo com retorno garantido.'}]},
  '2': {'Id': 2,
   'Nome': 'Marcus Lamb',
   'Idade': 37,
   'Agencia': 1001,
   'Conta': 80952,
   'Saldo': 1309.1,
   'Cartao': '**** **** **** 1507',
   'Limite': 840.15,
   'noticias': [{'Id': 2,
     'Mensagem': 'Marcus Lamb, proteja-se com nosso seguro de vida.'}]},
  '3': {'Id': 3,
   'Nome': 'Emily Woods',
   'Idade': 41,
   'Agencia': 1001,
   'Conta': 70464,
   'Saldo': 970.0,
   'Cartao': '**** **** **** 4691',
   'Limite': 903.74,
   'noticias': [{'Id': 3,
     'Mensagem': 'Emily Woods, proteja-se com nosso seguro de vida.'}]},
  '4': {'Id': 4,
   'Nome': 'Cameron Davis',
   'Idade': 50,
   'Agencia': 1001,
   'Conta': 65039,
   'Saldo': 553.9,
   'Cartao': '**** **** **** 6047',
   'L

# Fim